In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [8]:
df = pd.read_csv("../input/fake-news/train.csv")

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
###Drop Nan Values
df=df.dropna()

In [11]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [13]:
## Get the Dependent features
y=df['label']

In [14]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [15]:
X.shape

(18285, 4)

In [16]:
y.shape

(18285,)

In [17]:
import tensorflow as tf

In [18]:
tf.__version__

'2.6.4'

In [19]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [20]:
### Vocabulary size
voc_size=5000

**One Hot Representation**

In [22]:
messages=X.copy()

In [23]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [24]:
messages.reset_index(inplace=True)

In [25]:
import nltk
import re
from nltk.corpus import stopwords

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
#     print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

**Embedding Representation**

In [29]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[562, 3783, 73, 4571, 3069, 3204, 288, 3422, 1214, 4367],
 [2021, 1823, 3211, 3013, 3153, 4094, 1591],
 [2889, 4216, 3275, 205],
 [527, 3595, 1823, 3264, 4767, 1301],
 [3856, 3153, 171, 1210, 4150, 2994, 3153, 1508, 1210, 4423],
 [2183,
  2740,
  573,
  4623,
  359,
  2279,
  3546,
  2163,
  2156,
  2875,
  4497,
  984,
  4446,
  3046,
  1591],
 [1184, 3065, 4144, 2113, 2116, 2314, 4286, 4336, 1285, 4924, 2775],
 [469, 84, 3644, 1634, 3732, 2876, 2279, 2477, 1285, 4924, 2775],
 [4210, 2098, 3522, 311, 4931, 1975, 4157, 3978, 2279, 1833],
 [1820, 2368, 4908, 623, 4838, 2220, 3636, 4589],
 [4797, 4553, 4798, 3849, 2215, 3444, 2562, 44, 4452, 524, 3300],
 [3264, 1952, 3069, 1975, 2279, 3732],
 [1836, 2557, 2768, 224, 3454, 3247, 1792, 4737, 4881],
 [3711, 4586, 31, 1388, 1076, 4479, 3870, 1285, 4924, 2775],
 [544, 1520, 474, 1081, 3894, 1285, 4924, 2775],
 [2115, 2852, 1977, 3921, 63, 3686, 2855, 1697, 2093, 2268],
 [460, 725, 1823],
 [1895, 2100, 3580, 4975, 2279, 3349, 144, 1591],
 [37

In [30]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3422 1214 4367]
 [   0    0    0 ... 3153 4094 1591]
 [   0    0    0 ... 4216 3275  205]
 ...
 [   0    0    0 ... 1285 4924 2775]
 [   0    0    0 ... 3240  445 4339]
 [   0    0    0 ... 1886 2791  768]]


In [31]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  562,
       3783,   73, 4571, 3069, 3204,  288, 3422, 1214, 4367], dtype=int32)

In [32]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

2022-09-22 04:06:50.631450: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
len(embedded_docs),y.shape

(18285, (18285,))

In [35]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [36]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

**Model Training**

In [38]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

2022-09-22 04:08:09.707881: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
192/192 [==============================] - 11s 39ms/step - loss: 0.3064 - accuracy: 0.8524 - val_loss: 0.1880 - val_accuracy: 0.9203
Epoch 2/10
192/192 [==============================] - 7s 35ms/step - loss: 0.1333 - accuracy: 0.9491 - val_loss: 0.1941 - val_accuracy: 0.9238
Epoch 3/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0937 - accuracy: 0.9669 - val_loss: 0.2124 - val_accuracy: 0.9206
Epoch 4/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0649 - accuracy: 0.9791 - val_loss: 0.3163 - val_accuracy: 0.9147
Epoch 5/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0506 - accuracy: 0.9832 - val_loss: 0.3014 - val_accuracy: 0.9165
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0345 - accuracy: 0.9878 - val_loss: 0.3512 - val_accuracy: 0.9143
Epoch 7/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0249 - accuracy: 0.9918 - val_loss: 0.4553 - val_accuracy: 0.909

**Performance Metrics And Accuracy**

In [48]:
y_pred1=model1.predict(X_test)
classes_y=np.argmax(y_pred1,axis=1)

In [49]:
from sklearn.metrics import confusion_matrix

In [50]:
confusion_matrix(y_test,classes_y)

array([[3419,    0],
       [2616,    0]])

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,classes_y)

0.5665285832642917

In [53]:
from sklearn.metrics import classification_report
print(classification_report(y_test,classes_y))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
